In [1]:

import torch
print("torch:", torch.__version__, "cuda:", torch.version.cuda)
print("CUDA available:", torch.cuda.is_available())
print("Device:", torch.cuda.get_device_name(0) if torch.cuda.is_available() else "CPU")


torch: 2.8.0+cu128 cuda: 12.8
CUDA available: True
Device: NVIDIA GeForce RTX 3080


In [5]:
from transformers import pipeline, VideoMAEForVideoClassification
import accelerate
import scipy
import librosa as lr
import soundfile as sf
import numpy as np
import pandas as pd
import os
import tensorflow as tf
from datasets import load_dataset
from decord import VideoReader
from decord import cpu, gpu


# Video emotion detection
- training a model to detect the emotion present in a video or scene
- the scene may have people in it or maybe it could convey a mood based on the color
- I selected VideoMAE because it is very effcient with data meaning that it can be used when i dont have enough video data


In [6]:
# Load VideoMAE model
MAE_model = VideoMAEForVideoClassification.from_pretrained("MCG-NJU/videomae-base-finetuned-kinetics", attn_implementation="sdpa", dtype=torch.float16)
print("VideoMAE is loaded")

VideoMAE is loaded


# Preprocessing the Video data for emotion detection


In [7]:
# loading MSR-VTT dataset with the 'train_9k' and 'test_1k' splits
msrvtt_data_train = load_dataset("friedrichor/MSR-VTT", name="train_9k")
msrvtt_data_test = load_dataset("friedrichor/MSR-VTT", name="test_1k")

# displaying the caption and video segments of the dataset
row = msrvtt_data_train['train'][0]
print("Caption:", row['caption'])
print("Video segments:", row['video'])

Caption: ['a car is shown', 'a group is dancing', 'a man drives a vehicle through the countryside', 'a man drives down the road in an audi', 'a man driving a car', 'a man is driving a car', 'a man is driving down a road', 'a man is driving in a car as part of a commercial', 'a man is driving', 'a man riding the car speedly in a narrow road', 'a man showing the various features of a car', 'a man silently narrates his experience driving an audi', 'a person is driving his car around curves in the road', 'a person telling about a car', 'guy driving a car down the road', 'man talking about a car while driving', 'the man drives the car', 'the man driving the audi as smooth as possible', 'a man is driving', 'guy driving a car down the road']
Video segments: video0.mp4


### Reading in the videos from the dataset

In [11]:
# created to point to where the videos are stored in my local hugging face cache directory
cache_item = msrvtt_data_train.cache_files["test"][0]["filename"]  # usually the ZIP

# this is the cache folder 
base_dir = os.path.dirname(cache_item)  # cache folder for this dataset/config

vid_path = row["video"]

# making sure the video path is absolute
if not os.path.isabs(vid_path):
    vid_path = os.path.join(base_dir, vid_path)
    
### Reading in the videos from the dataset
def video_clipping(video_path, fps=2,max_frames=32):
    """Extract frames from a video file."""
    
    # vr is used create a VideoReader object to read the video file which allows for access to individual frames
    vr = VideoReader(video_path, ctx=cpu(0))
    
    # idx is used to create a list of frame indices to be extracted from the video or sampled uniformly across the video's duration
    idx = list(range(0, len(vr), max(1, len(vr)//max_frames)))[:max_frames]
    
    # frames is used to extract the frames from the video at the specified indices and convert them to a numpy array
    frames = vr.get_batch(idx).asnumpy()   # (T, H, W, 3) uint8
    
    return frames

# Testing the video_clipping function
clip = video_clipping(row["video"])


KeyError: 'test'